# Proyecto 1
# Universidad Galileo
## Ciencia de Datos en Python

### Mario Obed Morales Güitz
### 24006981

# Entorno de trabajo

## Importando las librerias que se utilizarán

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import warnings


In [ ]:
# Carga de Datos
sns.set_style("darkgrid")
d = np.load('proyecto_training_data.npy') 
df = pd.DataFrame(d)
df

# Separando Información

## atasets: entrenamiento(80 %) 
## validación y pruebas(20 %).

In [ ]:
Porcentaje = int(df[0].count()*.8)
Porcentaje

In [ ]:
Set_Entrenamiento = df.iloc[:Porcentaje]
Set_Entrenamiento = Set_Entrenamiento.set_axis(['SalePrice', 'OverallQual', '1stFLrSF','TotRmsAbvGrd', 
                            'YearBuilt', 'LotFrontage'], 
                    axis='columns')
Set_Entrenamiento

In [ ]:
Set_Pruebas = df.iloc[Porcentaje:]
Set_Pruebas = Set_Pruebas.set_axis(['SalePrice', 'OverallQual', '1stFLrSF','TotRmsAbvGrd', 'YearBuilt', 'LotFrontage'], 
                                   axis='columns')
Set_Pruebas

# Análisis Exploratorio de Datos

1. media
2. valor m´aximo
3. valor m´ınimo
4. rango(peak to peak, no el rango del tensor que por ser vector sabemos que es 1)
5. desviaci´on est´andar

In [ ]:
Media = Set_Entrenamiento.mean(axis=0)
Valor_Maximo =  Set_Entrenamiento.max(axis=0)
Valor_Minimo = Set_Entrenamiento.min(axis=0)
Desviacion_Estandar = Set_Entrenamiento.std(axis=0)
Rango = Valor_Maximo - Valor_Minimo

Resultados = ({'Media':pd.Series(Media,dtype=float), 'Valor_Maximo':pd.Series(Valor_Maximo,dtype=float), 'Valor_Minimo':pd.Series(Valor_Minimo,dtype=float),
             'Desviacion_Estandar':pd.Series(Desviacion_Estandar,dtype=float), 'Rango':pd.Series(Rango)})
dfAnalisis = pd.DataFrame(Resultados)
dfAnalisis


In [ ]:
Set_Entrenamiento.describe()

# Usando Seaborn para graficar

In [ ]:
# Quitamos las advertencias, pues es solo ajustes de tamaño
warnings.filterwarnings('ignore', category=UserWarning)

for i in Set_Entrenamiento.columns.values:
    ax = sns.displot(Set_Entrenamiento[i], edgecolor='k', color="#FFE4B5")
    plt.xlabel(i)
    plt.ylabel('Conteo')
    plt.gcf().set_size_inches(10, 6)  #  tamanio de la figura (ancho, alto)
    plt.show()


# Para cada variable independiente x

- Calcular el coeficiente de correlación entre x y y.
- Graficar x vs y(scatterplot) usando matplotlib.
- Colocar el coeficiente de correlación y colocarlo como parte del título de la gráfica.
- Basado en la gráfica y el coeficiente de correlación de cada par x,y elegir las 2 variables 
con más potencial predictivo es decir las 2 variables que presentan mayor correlación
entre dicha variable y la variable dependiente

In [ ]:
Correlaciones = []
variables = []
for i in Set_Entrenamiento.columns.values:
    Correlacion = (pd.DataFrame(Set_Entrenamiento[[i,'SalePrice']]).corr().iloc[0,1])
    Correlaciones.append(Correlacion)
    variables.append(i)
    plt.scatter(Set_Entrenamiento[i], Set_Entrenamiento['SalePrice'], edgecolor='k', color="#DAA520")
    plt.title(str(i)+' vs ' + str('SalePrice') + ' / Corr:  '+str((Correlacion)))
    plt.show()

In [ ]:
top_corr_variables = pd.DataFrame({'variables':variables, 
                                   'Correlaciones':Correlaciones})
top_corr_variables.sort_values(by='Correlaciones', ascending = False)[1:3]

In [ ]:
sns.pairplot(Set_Entrenamiento)

In [ ]:
Mejor_Potencial = pd.DataFrame({'Correlaciones':Correlaciones},index=[Set_Entrenamiento.columns.values])
Mejor_Potencial

# Crear una función para entrenar un modelo de Regresion Lineal

    6.1 Vector con la variable independiente x,
    6.2 Vector con la variable dependiente y,
    6.3 un entero epochs que indica por cuantas iteraciones entrenar el modelo.
    6.4 un entero imprimir error cada , que nos indica cada cuantas iteraciones queremos imprimir a través de print: el nu´mero de iteración, el error del modelo en esa iteración, si imprimir error cada = 10, se despliega en pantalla el error en las iteraciones: 10,20,30,40,50.
    6.5 escalar α(learning rate): es usado como parte de la expresión matemática para actualizar en cada iteración los parámetros del modelo

In [ ]:
class _regresion():
    #Constructor
    def __init__(self, x, y, epochs, imprimir_error_cada, learning_rate):
        self.x = x
        self.y = y
        self.b0 = 0
        self.b1 = 0
        self.epochs = epochs
        self.Error = []
        self.Modelo = {}  
        self.VectorY = np.reshape(self.y, (-1,1))
        self.VectorUnos = np.ones_like(self.x).reshape(-1,1)
        self.VectorXUnos = np.hstack([np.reshape(self.x,(-1,1)),self.VectorUnos])
        self.betas = np.reshape([[self.b1],[self.b0]],(-1,1))
      
        for i in range(epochs):
            self.y_hat = np.dot(self.VectorXUnos, self.betas)
            self.Errores = 1/(2*self.VectorY.shape[0]) * sum((self.VectorY - self.y_hat)**2)
            self.Error.append(self.Errores[0])
            if i%round(imprimir_error_cada) == 0:
                print('\nIteraciones: ', i, '    Error: ',self.Errores[0])
            self.gradiante = np.reshape(learning_rate*(1/self.VectorY.shape[0]*sum((self.y_hat - self.VectorY)*self.VectorXUnos)),(-1,1))
            self.betas = self.betas - self.gradiante
            self.Modelo[i] = [self.betas, self.y_hat, self.VectorY, self.x]
    
    #Modelo
    def modelo(self):
        return self.Modelo , np.array(self.Error)
    
    #Errores del Modelo
    @staticmethod
    def error(error):
        fig_1 = plt.figure(figsize = (9,7))
        ax = fig_1.add_axes([0.02, 0.02, 0.6, 0.6])
        ax.plot(error, color="#ADD8E6")
        ax.set_title("Errores del modelo")
        ax.set_xlabel("Numero de Iteraciones")
        ax.set_ylabel('Error')
        plt.show()
    
    #Evolución del Modelo
    @staticmethod
    def evolucion(Modelo, n):
        for i in list(Modelo.keys()):
            if i%round(n) == 0:
                fig_2 = plt.figure(figsize = (9,7))
                ax_2 = fig_2.add_axes([0.02, 0.02, 0.6, 0.6])
                y_hat = Modelo[i][1]
                y = Modelo[i][2]
                x = Modelo[i][3]
                ax_2.scatter(x , y, edgecolor='k', color="#8A2BE2")
                ax_2.plot(x, y_hat , linestyle ='solid', color ='r', label = 'Iteración:  %d' % i)
                ax_2.set_title("Iteración: "+str(i))
                ax_2.legend()
                ax_2.set_xlabel("X")
                ax_2.set_ylabel("Y")
                plt.show()
    
    #Método de Predicción 
    def prediccion(self, M):
        self.M = M
        self.VectorUnos = np.ones_like(self.M).reshape(-1,1)
        self.VectorXUnos = np.hstack([np.reshape(self.M,(-1,1)),self.VectorUnos])
        for i in range(self.epochs):
            self.Y_pred = np.dot(self.VectorXUnos, self.betas)
        return self.Y_pred
    
    #Compara Predicciones
    @staticmethod
    def compara_prediccion(Mpropio, Msklearn, Predecir, Modelo):
        MpropioResultado = Mpropio.prediccion(Predecir.reshape(-1,1))
        MsklearnResultado = Msklearn.predict(Predecir.reshape(-1,1)) 
        Promedio = (MpropioResultado + MsklearnResultado)/2
        fig_3 = plt.figure(figsize = (9,7))
        ax_3 = fig_3.add_axes([0.02, 0.02, 0.6, 0.6])
        for i in list(Modelo.columns.values):
            y = Modelo[i]
            ax_3.scatter(Predecir , y, edgecolor='k', color="#FAEBD7")
            ax_3.plot(Predecir, MpropioResultado, color="#FFEBCD", label = '_regresion: ')
            ax_3.plot(Predecir, MsklearnResultado, color="#FFFACD", label = 'Msklearn: ')
            ax_3.plot(Predecir, Promedio, color="orange", label = 'Promedio: ')
            ax_3.set_title("Comparativa")
            ax_3.legend()
            ax_3.set_xlabel("X")
            ax_3.set_ylabel('Y')
            plt.show()
        return MpropioResultado, MsklearnResultado, Promedio
    
    #Compara Errores
    @staticmethod
    def compara_errores(modelos, modelos2, prueba1, prueba2, set_prueba):
        labels = ['OverallQual', '1stFLrSF']
        Error_regresion = metrics.mean_squared_error(modelos[0], prueba1)
        Error_Msklearn = metrics.mean_squared_error(modelos[1], prueba1)
        Error_Promedio = metrics.mean_squared_error(modelos[2], prueba1)
        
        Error2_regresion = metrics.mean_squared_error(modelos2[0], prueba2)
        Error2_Msklearn = metrics.mean_squared_error(modelos2[1], prueba2)
        Error2_Promedio = metrics.mean_squared_error(modelos2[2], prueba2)
        
        Errores_regresion = [Error_regresion,Error2_regresion]
        Errores_Msklearn = [Error_Msklearn,Error2_Msklearn]
        Errores_Promedio = [Error_Promedio,Error2_Promedio]
        
        T = np.arange(len(labels))
        fig_4 = plt.figure(figsize = (9,7))
        ax = fig_4.add_axes([0.01, 0.01, 0.6, 0.6])
        ax.set_xticks(T + 0.90/3)
        ax.set_xticklabels(labels)
        ax.bar(T, Errores_regresion, width=0.30,  edgecolor='k', color="#708090", label="_regresion")
        ax.bar(T+ 0.30, Errores_Msklearn, width=0.30,  edgecolor='k', color="#E6E6FA", label="Msklearn")
        ax.bar(T+ 0.60, Errores_Promedio, width=0.30, edgecolor='k', color="#F0FFF0", label="Promedio")
        ax.legend()
        ax.set_title("Comparativa")
        plt.show()

## Entrenando modelos de regresión - con OverallQual

In [ ]:
NPSales = np.reshape(np.array(Set_Entrenamiento['SalePrice']), (-1, 1))
NPOverallQual = np.reshape(np.array(Set_Entrenamiento['OverallQual']), (-1, 1))
NPFirst_Floor_square_feet = np.reshape(np.array(Set_Entrenamiento['1stFLrSF']), (-1, 1))
Epoch = 5001
Imprimir = 1000
Escalar = 0.01
RegresionModelo1 = _regresion(NPOverallQual, NPSales, Epoch, Imprimir, Escalar)
Rmodelos, Rerrores = RegresionModelo1.modelo()
_regresion.error(Rerrores)
_regresion.evolucion(Rmodelos,Imprimir)

## Entrenando modelos de regresión - con 1stFlrSF

In [ ]:
Epoch = 51
Imprimir = 10
Escalar = 0.0000001
RegresionModelo2 = _regresion(NPFirst_Floor_square_feet, NPSales, Epoch, Imprimir, Escalar)
Rmodelos2, Rerrores2 = RegresionModelo2.modelo()
_regresion.error(Rerrores2)
_regresion.evolucion(Rmodelos2,Imprimir)

## Predicciones modelo propio vs SK-Learn

In [ ]:
Sci_NPOverallQual = LinearRegression()
Sci_NPOverallQual.fit(NPOverallQual, NPSales)
Sci_NPFirst_Floor_square_feet = LinearRegression()
Sci_NPFirst_Floor_square_feet.fit(NPFirst_Floor_square_feet, NPSales)

## Predicciones con modelo propio, Scikit Learn y Promedio

In [ ]:
Pruebas_OverallQual = np.reshape(np.array(Set_Pruebas['OverallQual']), (-1, 1))
Pruebas_Floor_square_feet = np.reshape(np.array(Set_Pruebas['1stFLrSF']), (-1, 1))

## Comparando modelos
    Modelo 1 - Modelo Propio
    Modelo 2 - Modelo Scikit Learn
    Modelo 3 - Modelo Promedio

In [ ]:
modelos_OverallQual = _regresion.compara_prediccion(RegresionModelo1, Sci_NPOverallQual, Pruebas_OverallQual, Set_Pruebas)
modelos_1stFlrSF = _regresion.compara_prediccion(RegresionModelo2, Sci_NPFirst_Floor_square_feet, Pruebas_Floor_square_feet, Set_Pruebas)

## Comparativa de errores de modelos

In [ ]:
_regresion.compara_errores(modelos_OverallQual, modelos_1stFlrSF, Pruebas_OverallQual, Pruebas_Floor_square_feet, Set_Pruebas)

## Conclusiones

- OverallQual: todos los modelos presentan un error muy similar, aproximadamente el 20% de los datos. Sin embargo, la regresión específicamente desarrollada para esta tarea resulta ser el modelo con el menor error. 
- 1stFlrSF: el modelo de regresión de scikit-learn sobresale como el mejor, mostrando un error significativamente menor en comparación con el modelo generado para esta tarea.